In [2]:
conda install tensor

Solving environment: ...working... failed with initial frozen solve. Retrying with flexible solve.
Solving environment: ...working... failed with initial frozen solve. Retrying with flexible solve.

Note: you may need to restart the kernel to use updated packages.



PackagesNotFoundError: The following packages are not available from current channels:

  - tensor

Current channels:

  - https://repo.anaconda.com/pkgs/main/win-64
  - https://repo.anaconda.com/pkgs/main/noarch
  - https://repo.anaconda.com/pkgs/r/win-64
  - https://repo.anaconda.com/pkgs/r/noarch
  - https://repo.anaconda.com/pkgs/msys2/win-64
  - https://repo.anaconda.com/pkgs/msys2/noarch

To search for alternate channels that may provide the conda package you're
looking for, navigate to

    https://anaconda.org

and use the search bar at the top of the page.




In [3]:
def _sharpness_image(image: TensorLike, factor: Number) -> tf.Tensor:
    """Implements Sharpness function from PIL using TF ops."""
    orig_image = image
    image_dtype = image.dtype
    image_channels = image.shape[-1]
    image = tf.cast(image, tf.float32)

    # SMOOTH PIL Kernel.
    kernel = (
        tf.constant(
            [[1, 1, 1], [1, 5, 1], [1, 1, 1]], dtype=tf.float32, shape=[3, 3, 1, 1]
        )
        / 13.0
    )
    kernel = tf.tile(kernel, [1, 1, image_channels, 1])

    # Apply kernel channel-wise.
    degenerate = tf.nn.depthwise_conv2d(
        image, kernel, strides=[1, 1, 1, 1], padding="VALID", dilations=[1, 1]
    )
    degenerate = tf.cast(degenerate, image_dtype)

    # For the borders of the resulting image, fill in the values of the original image.
    mask = tf.ones_like(degenerate)
    padded_mask = tf.pad(mask, [[0, 0], [1, 1], [1, 1], [0, 0]])
    padded_degenerate = tf.pad(degenerate, [[0, 0], [1, 1], [1, 1], [0, 0]])
    result = tf.where(tf.equal(padded_mask, 1), padded_degenerate, orig_image)

    # Blend the final result.
    blended = blend(result, orig_image, factor)
    return tf.cast(blended, image_dtype)


@tf.function
def sharpness(image: TensorLike, factor: Number, name: Optional[str] = None) -> tf.Tensor:
    """Change sharpness of image(s).
    Args:
      image: A tensor of shape
          `(num_images, num_rows, num_columns, num_channels)` (NHWC), or
          `(num_rows, num_columns, num_channels)` (HWC)
      factor: A floating point value or Tensor above 0.0.
      name: The name of the op.
    Returns:
      Image(s) with the same type and shape as `images`, sharper.
    """
    with tf.name_scope(name or "sharpness"):
        image_dims = tf.rank(image)
        image = to_4D_image(image)
        image = _sharpness_image(image, factor=factor)
        return from_4D_image(image, image_dims)

NameError: name 'TensorLike' is not defined